In [48]:
import json
import os
import openai
from dotenv import load_dotenv
import anthropic

In [53]:
# Read QA questions from the json file
with open('shuffled_QA_pairs_econ_part2.json') as f:
    data = json.load(f)

# Load the .env file
load_dotenv()
client = openai.OpenAI()
claude_client = anthropic.Anthropic()

In [14]:
data[0]['choices']

['Nonexcludable goods and services include police protection and most roads, as it is challenging to prevent individuals from benefiting from these once they are provided. Public primary education also falls into this category, as it is generally accessible to all, despite the existence of private options that may be excludable. In contrast, streaming music services, cell phone service, and toll roads are excludable, as access is limited to paying customers or users.',
 'Among the options provided, roads and public primary education are considered nonexcludable. Most roads are nonexcludable because, once constructed, it is difficult to prevent individuals from using them, although toll roads can create exceptions. Public primary education is generally nonexcludable as it is available to all children regardless of payment. In contrast, police protection is excludable since access can be limited to those who pay for private security services. Similarly, streaming music services like Siri

In [75]:
from tqdm import tqdm
from collections import defaultdict
log = defaultdict(dict)

for generated_qa in tqdm(data, desc="Generating answers"):
    question = generated_qa['question']
    correct_answer = generated_qa['correct_answer']
    choices = generated_qa['mcq']

    prompt = f"""
    Question: {choices}
    Which of the above choices best answers the question? Give a reason for your choice first, then give your choice.
    Reply a JSON object with the following format:
    {{
        "reason": "string",
        "choice": "int",
    }}
    """
    temperature = 0.0

    response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a student answering a textbook problem."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
    
    answer_json = json.loads(response.choices[0].message.content)

    log[question]['reason'] = answer_json['reason']
    log[question]['gpt_choice'] = answer_json['choice']
    log[question]['correct_choice'] = correct_answer

Generating answers: 100%|██████████| 161/161 [04:41<00:00,  1.75s/it]


In [76]:
correct_count = 0
total_count = len(log)

for _, answers in log.items():
    if answers['gpt_choice'] == answers['correct_choice']:
        correct_count += 1

percentage_correct = (correct_count / total_count) * 100
print(f"Percentage of correct choices: {percentage_correct:.2f}%")

Percentage of correct choices: 8.70%


In [80]:
log[list(log.keys())[0]]

{'reason': 'Choice D best answers the question because it correctly identifies police protection and most roads as nonexcludable, acknowledging the difficulty in preventing individuals from benefiting from these services once provided. It also correctly notes that public primary education is generally nonexcludable, while distinguishing that streaming music services, cell phone service, and toll roads are excludable due to their access restrictions.',
 'gpt_choice': 4,
 'correct_choice': 3}

In [55]:
anthropic_log = defaultdict(dict)

for generated_qa in tqdm(data, desc="Generating answers"):
    question = generated_qa['question']
    correct_answer = generated_qa['correct_answer']
    choices = generated_qa['choices']

    prompt = f"""
    Question: {question}
    Choice 1: {choices[0]}
    Choice 2: {choices[1]}
    Choice 3: {choices[2]}
    Choice 4: {choices[3]}
    Which of the above definitions best answers the question? Give a reason for your choice first, then give your choice.
    Reply a JSON object with the following format:
    {{
        "reason": "string",
        "choice": "int",
    }}
    """
    temperature = 0.0

    response = claude_client.messages.create(
                model="claude-3-5-sonnet-20241022",
                messages=[
                    {"role": "system", "content": "You are a student answering a textbook problem."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=1000,
            )
    
    answer_json = json.loads(response.choices[0].message.content)

    anthropic_log[question]['reason'] = answer_json['reason']
    anthropic_log[question]['gpt_choice'] = answer_json['choice']
    anthropic_log[question]['correct_choice'] = correct_answer

    print(anthropic_log)
    break

Generating answers:   0%|          | 0/161 [00:21<?, ?it/s]


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}